In [147]:
%pip install pandas
import pandas as pd




[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [148]:
stats = pd.read_csv("teamCsvFiles/OaklandAthletics.csv", encoding='latin1')
stats.fillna(0, inplace=True)

In [219]:
stats

,gameId,teamId,opponentId,homeTeam?,playerInfoId,playerFullName,battingRuns,battingDoubles,battingTriples,battingHomeRuns,...,pitchingAtBats,pitchingStolenBases,pitchingNumberOfPitches,pitchingInningsPitched,pitchingWins,pitchingLosses,pitchingHolds,pitchingBlownSaves,pitchingEarnedRuns,pitchingPitchesThrown
0,717538,133,145,True,542208,Carlos Pérez,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,717538,133,145,True,621112,Paul Blackburn,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,717538,133,145,True,669674,Sam Long,0.0,0.0,0.0,0.0,...,1.0,0.0,10.0,0.1,0.0,0.0,0.0,0.0,0.0,10.0
3,717538,133,145,True,667670,Brent Rooker,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,717538,133,145,True,670329,Rico Garcia,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781,747949,133,137,True,660906,Francisco Pérez,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3782,747949,133,137,True,694977,Nate Nankil,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3783,747949,133,137,True,691740,Daniel Susac,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3784,747949,133,137,True,676206,Freddy Tarnok,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [217]:
teamStats = stats.groupby(['playerFullName']).sum().reset_index()

# Drop the 'teamId' column
teamStats = teamStats.drop(['teamId','gameId','opponentId','homeTeam?','playerInfoId'], axis=1)

In [218]:
teamStats

,playerFullName,battingRuns,battingDoubles,battingTriples,battingHomeRuns,battingStrikeOuts,battingBaseOnBalls,battingHits,battingAtBats,battingStolenBases,...,pitchingAtBats,pitchingStolenBases,pitchingNumberOfPitches,pitchingInningsPitched,pitchingWins,pitchingLosses,pitchingHolds,pitchingBlownSaves,pitchingEarnedRuns,pitchingPitchesThrown
0,Aaron Brooks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abraham Toro,4.0,0.0,0.0,1.0,5.0,4.0,11.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Adrián Martínez,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,128.0,5.0,542.0,30.1,0.0,2.0,1.0,0.0,18.0,542.0
3,Aledmys Díaz,11.0,11.0,1.0,2.0,27.0,8.0,34.0,129.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alex Wood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36.0,0.0,80.0,8.2,0.0,0.0,0.0,0.0,7.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Will Simpson,0.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118,Yohel Pozo,1.0,0.0,0.0,1.0,2.0,1.0,1.0,14.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119,Zach Jackson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,1.0,51.0,4.1,0.0,1.0,2.0,1.0,6.0,51.0
120,Zach Neal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,93.0,0.0,426.0,22.5,1.0,1.0,0.0,0.0,17.0,426.0
